# Classificando Notícias - RNNs
As redes neurais recorrentes (RNNs) são uma classe de redes neurais que é poderosa para modelar sequências de dados. Elas são amplamente utilizadas para tarefas de processamento de linguagem natural, como tradução automática, geração de texto e classificação de texto. Neste notebook, vamos usar uma RNN para classificar notícias em categorias.

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import re
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from wordcloud import STOPWORDS
from nltk.stem import WordNetLemmatizer

import torch
from torch import nn

[nltk_data] Downloading package wordnet to /Users/zfab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Carregando os Dados

In [5]:
df_raw = pd.read_csv('../data/bbc-text.csv', sep=',')
df_raw.sample(5)

,category,text
1707,politics,kilroy-silk quits shameful ukip ex-chat show...
196,entertainment,versace art portfolio up for sale the art coll...
1222,tech,us top of supercomputing charts the us has pus...
1672,sport,real will finish abandoned match real madrid a...
1880,tech,doors open at biggest gadget fair thousands of...
